In [6]:
# Importing essential libraries

import numpy as np
import pandas as pd
import nltk
import spacy
import string
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer , CountVectorizer

In [2]:
# Loading dataset for postitve and negative review

pos_review = pd.read_csv('pos.txt',encoding='latin-1',header=None,sep='\n')
pos_review['sentiment']=1
pos_review.rename(columns={0:'review'},inplace=True)
neg_review = pd.read_csv('negative.txt',encoding='latin-1',header=None,sep='\n')
neg_review['sentiment']=0
neg_review.rename(columns={0:'review'},inplace=True)

In [3]:
pos_review

,review,sentiment
0,the rock is destined to be the 21st century's ...,1
1,"the gorgeously elaborate continuation of "" the...",1
2,effective but too-tepid biopic,1
3,if you sometimes like to go to the movies to h...,1
4,"emerges as something rare , an issue movie tha...",1
...,...,...
5326,both exuberantly romantic and serenely melanch...,1
5327,mazel tov to a film about a family's joyous li...,1
5328,standing in the shadows of motown is the best ...,1
5329,it's nice to see piscopo again after all these...,1


In [4]:
neg_review

,review,sentiment
0,"simplistic , silly and tedious.",0
1,"it's so laddish and juvenile , only teenage bo...",0
2,exploitative and largely devoid of the depth o...,0
3,[garbus] discards the potential for pathologic...,0
4,a visually flashy but narratively opaque and e...,0
...,...,...
5326,a terrible movie that some people will neverth...,0
5327,there are many definitions of 'time waster' bu...,0
5328,"as it stands , crocodile hunter has the hurrie...",0
5329,the thing looks like a made-for-home-video qui...,0


In [11]:
all_data = pd.concat([pos_review,neg_review],axis=0,ignore_index=True)
all_data.head()

,review,sentiment
0,the rock is destined to be the 21st century's ...,1
1,"the gorgeously elaborate continuation of "" the...",1
2,effective but too-tepid biopic,1
3,if you sometimes like to go to the movies to h...,1
4,"emerges as something rare , an issue movie tha...",1


In [12]:
all_data.tail()

,review,sentiment
10657,a terrible movie that some people will neverth...,0
10658,there are many definitions of 'time waster' bu...,0
10659,"as it stands , crocodile hunter has the hurrie...",0
10660,the thing looks like a made-for-home-video qui...,0
10661,"enigma is well-made , but it's just too dry an...",0


In [13]:
df = all_data[['review']]
df.head()

,review
0,the rock is destined to be the 21st century's ...
1,"the gorgeously elaborate continuation of "" the..."
2,effective but too-tepid biopic
3,if you sometimes like to go to the movies to h...
4,"emerges as something rare , an issue movie tha..."


# Text Preprocessing

### Converting text to lower case

In [15]:
df["text_lower"] = df["review"].str.lower()
df.head()

,review,text_lower
0,the rock is destined to be the 21st century's ...,the rock is destined to be the 21st century's ...
1,"the gorgeously elaborate continuation of "" the...","the gorgeously elaborate continuation of "" the..."
2,effective but too-tepid biopic,effective but too-tepid biopic
3,if you sometimes like to go to the movies to h...,if you sometimes like to go to the movies to h...
4,"emerges as something rare , an issue movie tha...","emerges as something rare , an issue movie tha..."


### Removing Stopwords

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/Adit/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
# all stopwords in nltk library
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [18]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df["text_wo_stop"] = df["text_lower"].apply(lambda text: remove_stopwords(text))
df.head()

,review,text_lower,text_wo_stop
0,the rock is destined to be the 21st century's ...,the rock is destined to be the 21st century's ...,"rock destined 21st century's new "" conan "" he'..."
1,"the gorgeously elaborate continuation of "" the...","the gorgeously elaborate continuation of "" the...","gorgeously elaborate continuation "" lord rings..."
2,effective but too-tepid biopic,effective but too-tepid biopic,effective too-tepid biopic
3,if you sometimes like to go to the movies to h...,if you sometimes like to go to the movies to h...,"sometimes like go movies fun , wasabi good pla..."
4,"emerges as something rare , an issue movie tha...","emerges as something rare , an issue movie tha...","emerges something rare , issue movie that's ho..."


### Remove Punctuations

In [19]:
PUNCT_TO_REMOVE = string.punctuation
print (PUNCT_TO_REMOVE)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [20]:
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df["text_wo_punct"] = df["text_wo_stop"].apply(lambda text: remove_punctuation(text))
df.head()

,review,text_lower,text_wo_stop,text_wo_punct
0,the rock is destined to be the 21st century's ...,the rock is destined to be the 21st century's ...,"rock destined 21st century's new "" conan "" he'...",rock destined 21st centurys new conan hes go...
1,"the gorgeously elaborate continuation of "" the...","the gorgeously elaborate continuation of "" the...","gorgeously elaborate continuation "" lord rings...",gorgeously elaborate continuation lord rings ...
2,effective but too-tepid biopic,effective but too-tepid biopic,effective too-tepid biopic,effective tootepid biopic
3,if you sometimes like to go to the movies to h...,if you sometimes like to go to the movies to h...,"sometimes like go movies fun , wasabi good pla...",sometimes like go movies fun wasabi good plac...
4,"emerges as something rare , an issue movie tha...","emerges as something rare , an issue movie tha...","emerges something rare , issue movie that's ho...",emerges something rare issue movie thats hone...


### Removing rare words

In [21]:
from collections import Counter
cnt = Counter()
for text in df["text_wo_punct"].values:
    for word in text.split():
        cnt[word] += 1
# Printing 10 most common words        
cnt.most_common(10)

# Some of these words such as like,good are essential for sentiment analysis hence not removing them

[('film', 1453),
 ('movie', 1270),
 ('one', 727),
 ('like', 721),
 ('story', 477),
 ('much', 386),
 ('even', 382),
 ('good', 377),
 ('comedy', 359),
 ('time', 341)]

In [25]:
# Printing 20 most rare words
n_rare_words = 20
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
print(RAREWORDS)

{'definitions', 'shouting', 'hurried', 'roller', 'rodan', 'coaster', 'dozing', 'pamelas', 'juliet', 'agile', 'startled', 'stevenons', 'madeforhomevideo', 'commenting', 'chore', 'waster', '1959', 'uncertainty', 'principle', 'discount'}


In [26]:
def remove_rarewords(text):
    """custom function to remove the rare words"""
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

df["text_wo_stopfreqrare"] = df["text_wo_punct"].apply(lambda text: remove_rarewords(text))
df.head()

,review,text_lower,text_wo_stop,text_wo_punct,text_wo_stopfreqrare
0,the rock is destined to be the 21st century's ...,the rock is destined to be the 21st century's ...,"rock destined 21st century's new "" conan "" he'...",rock destined 21st centurys new conan hes go...,rock destined 21st centurys new conan hes goin...
1,"the gorgeously elaborate continuation of "" the...","the gorgeously elaborate continuation of "" the...","gorgeously elaborate continuation "" lord rings...",gorgeously elaborate continuation lord rings ...,gorgeously elaborate continuation lord rings t...
2,effective but too-tepid biopic,effective but too-tepid biopic,effective too-tepid biopic,effective tootepid biopic,effective tootepid biopic
3,if you sometimes like to go to the movies to h...,if you sometimes like to go to the movies to h...,"sometimes like go movies fun , wasabi good pla...",sometimes like go movies fun wasabi good plac...,sometimes like go movies fun wasabi good place...
4,"emerges as something rare , an issue movie tha...","emerges as something rare , an issue movie tha...","emerges something rare , issue movie that's ho...",emerges something rare issue movie thats hone...,emerges something rare issue movie thats hones...


### Removing digits


In [27]:
def remove_digits(text):
    """function to remove digits"""
    return " ".join([word for word in str(text).split() if not word.isdigit()])


df["text_wo_digits"] = df["text_wo_stopfreqrare"].apply(lambda text: remove_rarewords(text))
df.head()

,review,text_lower,text_wo_stop,text_wo_punct,text_wo_stopfreqrare,text_wo_digits
0,the rock is destined to be the 21st century's ...,the rock is destined to be the 21st century's ...,"rock destined 21st century's new "" conan "" he'...",rock destined 21st centurys new conan hes go...,rock destined 21st centurys new conan hes goin...,rock destined 21st centurys new conan hes goin...
1,"the gorgeously elaborate continuation of "" the...","the gorgeously elaborate continuation of "" the...","gorgeously elaborate continuation "" lord rings...",gorgeously elaborate continuation lord rings ...,gorgeously elaborate continuation lord rings t...,gorgeously elaborate continuation lord rings t...
2,effective but too-tepid biopic,effective but too-tepid biopic,effective too-tepid biopic,effective tootepid biopic,effective tootepid biopic,effective tootepid biopic
3,if you sometimes like to go to the movies to h...,if you sometimes like to go to the movies to h...,"sometimes like go movies fun , wasabi good pla...",sometimes like go movies fun wasabi good plac...,sometimes like go movies fun wasabi good place...,sometimes like go movies fun wasabi good place...
4,"emerges as something rare , an issue movie tha...","emerges as something rare , an issue movie tha...","emerges something rare , issue movie that's ho...",emerges something rare issue movie thats hone...,emerges something rare issue movie thats hones...,emerges something rare issue movie thats hones...


### Lemmatization

In [31]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Adit/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [32]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

df["text_lemmatized"] = df["text_wo_digits"].apply(lambda text: lemmatize_words(text))
df.head()

,review,text_lower,text_wo_stop,text_wo_punct,text_wo_stopfreqrare,text_wo_digits,text_lemmatized
0,the rock is destined to be the 21st century's ...,the rock is destined to be the 21st century's ...,"rock destined 21st century's new "" conan "" he'...",rock destined 21st centurys new conan hes go...,rock destined 21st centurys new conan hes goin...,rock destined 21st centurys new conan hes goin...,rock destine 21st centurys new conan he go mak...
1,"the gorgeously elaborate continuation of "" the...","the gorgeously elaborate continuation of "" the...","gorgeously elaborate continuation "" lord rings...",gorgeously elaborate continuation lord rings ...,gorgeously elaborate continuation lord rings t...,gorgeously elaborate continuation lord rings t...,gorgeously elaborate continuation lord ring tr...
2,effective but too-tepid biopic,effective but too-tepid biopic,effective too-tepid biopic,effective tootepid biopic,effective tootepid biopic,effective tootepid biopic,effective tootepid biopic
3,if you sometimes like to go to the movies to h...,if you sometimes like to go to the movies to h...,"sometimes like go movies fun , wasabi good pla...",sometimes like go movies fun wasabi good plac...,sometimes like go movies fun wasabi good place...,sometimes like go movies fun wasabi good place...,sometimes like go movie fun wasabi good place ...
4,"emerges as something rare , an issue movie tha...","emerges as something rare , an issue movie tha...","emerges something rare , issue movie that's ho...",emerges something rare issue movie thats hone...,emerges something rare issue movie thats hones...,emerges something rare issue movie thats hones...,emerges something rare issue movie thats hones...


In [33]:
X_train,X_test,y_train,y_test = train_test_split(df['text_lemmatized'].values,all_data['sentiment'].values,test_size=0.2,random_state=365)

In [34]:
train_data = pd.DataFrame({'review':X_train , 'sentiment':y_train})
test_data = pd.DataFrame({'review':X_test , 'sentiment':y_test})

In [35]:
train_data

,review,sentiment
0,movie amateurish minor treat,1
1,unfortunately heartbreak hospital want convey ...,0
2,australia land beyond time enjoyable big movie...,1
3,mr deed sure give lot laugh simple sweet roman...,1
4,sort burly action flick one coincidence pummel...,0
...,...,...
8524,whats next rob schneider dana carvey sarah mic...,0
8525,result something quite fresh delightful,1
8526,despite hoffman best effort wilson remain sile...,0
8527,unusually dryeyed even analytical approach mat...,1
